# Example LnL Computation


In [ ]:
from lnl_computer.cosmic_integration.mcz_grid import McZGrid
from lnl_computer.cosmic_integration.star_formation_paramters import (
    DEFAULT_DICT,
    draw_star_formation_samples
)
from lnl_computer.observation.mock_observation import MockObservation
from compas_python_utils.cosmic_integration.binned_cosmic_integrator.bbh_population import \
    generate_mock_bbh_population_file
from collections import namedtuple
import os
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

MOCK_DATA_TYPE = namedtuple('MockData', ['compas_h5_path', 'obs', 'sf_sample'])

TMPDIR = 'out_temp'
DURATION = 5


def generate_mock_data(tmpdir=TMPDIR, n_systems=2000):
    os.makedirs(tmpdir, exist_ok=True)
    mock_compas_h5 = generate_mock_bbh_population_file(
        n_systems=int(n_systems), 
        filename=f"{tmpdir}/mock_compas.h5",
    )
    mock_obs = MockObservation.from_compas_h5(
        mock_compas_h5, 
        duration=DURATION, 
        cosmological_parameters=DEFAULT_DICT
    )
    return MOCK_DATA_TYPE(mock_compas_h5, mock_obs, DEFAULT_DICT)




def compute_lnls_for_param(obs, param='mu_z', n_samples=10, ):
    lnls = []
    lnl_kwgs = dict(
        mcz_obs=obs,
        compas_h5_path=MOCK_DATA.compas_h5_path,
        save_plots=True,
        outdir=TMPDIR,
    )
    samples = draw_star_formation_samples(n_samples, parameters=param, grid=True, as_list=True)
    for sf_sample in samples:
        lnl, _ = McZGrid.lnl(sf_sample=sf_sample, **lnl_kwgs)
        lnls.append(lnl)
    s = [list(sf_sample.values())[0] for sf_sample in samples]
    return s, lnls


MOCK_DATA = generate_mock_data()

## Mock Observations

In [ ]:
N = 10
sigma0s, sigma0_lnls = compute_lnls_for_param(MOCK_DATA.obs, 'sigma_0', N)
muzs, muz_lnls = compute_lnls_for_param(MOCK_DATA.obs, 'mu_z', N)
aSFs, aSF_lnls = compute_lnls_for_param(MOCK_DATA.obs, 'aSF', N)
dSFs, dSF_lnls = compute_lnls_for_param(MOCK_DATA.obs, 'dSF', N)

In [ ]:

def plot_lnls(param, values, lnls, ax, show_trues=False):
    ax.plot(values, lnls)
    ylim = ax.get_ylim()
    if show_trues:
        ax.vlines(MOCK_DATA.sf_sample[param], *ylim, colors='red', linestyles='dashed')
    ax.set_ylim(ylim)
    ax.set_xlabel(param)
    ax.set_yticks([])

fig, axs = plt.subplots(1, 4, figsize=(12, 2))
plot_lnls('sigma_0', sigma0s, sigma0_lnls, axs[0], show_trues=True)
plot_lnls('mu_z', muzs, muz_lnls, axs[1], show_trues=True)
plot_lnls('aSF', aSFs, aSF_lnls, axs[2], show_trues=True)
plot_lnls('dSF', dSFs, dSF_lnls, axs[3], show_trues=True)
axs[0].set_ylabel('lnL')
plt.tight_layout()
fig.savefig(f"{TMPDIR}/mockobs_lnls.png")

![](out_temp/mockobs_lnls.png)

## Using LVK Observations

In [ ]:
from lnl_computer.observation import load_observation

lvk_obs = load_observation(
    'LVK',
    pastro_threshold=0.95,
    observing_runs=["O3a", "O3b"],
    filter_valid_mcz=True,
)

N = 10
sigma0s, lvk_sigma0_lnls = compute_lnls_for_param(lvk_obs, 'sigma_0', N)
muzs, lvk_muz_lnls = compute_lnls_for_param(lvk_obs, 'mu_z', N)
aSFs, lvk_aSF_lnls = compute_lnls_for_param(lvk_obs, 'aSF', N)
dSFs, lvk_dSF_lnls = compute_lnls_for_param(lvk_obs, 'dSF', N)

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12, 2))
plot_lnls('sigma_0', sigma0s, lvk_sigma0_lnls, axs[0])
plot_lnls('mu_z', muzs, lvk_muz_lnls, axs[1])
plot_lnls('aSF', aSFs, lvk_aSF_lnls, axs[2])
plot_lnls('dSF', dSFs, lvk_dSF_lnls, axs[3])
axs[0].set_ylabel('lnL')
plt.tight_layout()
fig.savefig(f"{TMPDIR}/lvk_lnls.png")

![](out_temp/lvk_lnls.png)